# LSTM for Part-of-Speech Tagging

In this section, we will use an LSTM to predict part-of-speech tags for words. What exactly is part-of-speech tagging?

Part of speech tagging is the process of determining the *category* of a word from the words in its surrounding context. You can think of part of speech tagging as a way to go from words to their [Mad Libs](https://en.wikipedia.org/wiki/Mad_Libs#Format) categories. Mad Libs are incomplete short stories that have many words replaced by blanks. Each blank has a specified word-category, such as `"noun"`, `"verb"`, `"adjective"`, and so on. One player asks another to fill in these blanks (prompted only by the word-category) until they have created a complete, silly story of their own. Here is an example of such categories:

```text
Today, you'll be learning how to [verb]. It may be a [adjective] process, but I think it will be rewarding! 
If you want to take a break you should [verb] and treat yourself to some [plural noun].
```
... and a set of possible words that fall into those categories:
```text
Today, you'll be learning how to code. It may be a challenging process, but I think it will be rewarding! 
If you want to take a break you should stretch and treat yourself to some puppies.
```


### Why Tag Speech?

Tagging parts of speech is often used to help disambiguate natural language phrases because it can be done quickly and with high accuracy. It can help answer: what subject is someone talking about? Tagging can be used for many NLP tasks like creating new sentences using a sequence of tags that make sense together, filling in a Mad Libs style game, and determining correct pronunciation during speech synthesis. It is also used in information retrieval, and for word disambiguation (ex. determining when someone says *right* like the direction versus *right* like "that's right!").

---


### Preparing the Data

Now, we know that neural networks do not do well with words as input and so our first step will be to prepare our training data and map each word to a numerical value. 

We start by creating a small set of training data, you can see that this is a few simple sentences broken down into a list of words and their corresponding word-tags. Note that the sentences are turned into lowercase words using `lower()` and then split into separate words using `split()`, which splits the sentence by whitespace characters.

#### Words to indices

Then, from this training data, we create a dictionary that maps each unique word in our vocabulary to a numerical value; a unique index `idx`. We do the same for each word-tag, for example: a noun will be represented by the number `1`.

In [18]:
# import resources
from typing import List, Tuple, Dict
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt

%matplotlib inline

In [26]:
Device: str = 'cuda' if torch.cuda.is_available() else 'cpu'

In [19]:
# training sentences and their corresponding word-tags
DataType = List[Tuple[List[str], List[str]]]

training_data: DataType = [
    ("The cat ate the cheese".lower().split(), ["DET", "NN", "V", "DET", "NN"]),
    ("She read that book".lower().split(), ["NN", "V", "DET", "NN"]),
    ("The dog loves art".lower().split(), ["DET", "NN", "V", "NN"]),
    ("The elephant answers the phone".lower().split(), ["DET", "NN", "V", "DET", "NN"])
]

# create a dictionary that maps words to indices
word2idx: Dict[str, int]  = {}

for sent, tags in training_data:
    for word in sent:
        # Checks and only inserts the word in 
        # dictionary if it is not prepsent already.
        # Dictionary must have a single occurrence of 
        # a given word from the training data set.
        if word not in word2idx:
            # Ensures, that a word to be inserted in 
            # the dictionary gets an index according to 
            # the point in time, in which it is being 
            # inserted. For the very starting word length 
            # of the dictionary is 0, for the second word 
            # length is 1 because the dictionary already 
            # has the first word and so on.
            word2idx[word] = len(word2idx)

# create a dictionary that maps tags to indices
tag2idx: Dict[str, int] = {"DET": 0, "NN": 1, "V": 2}

Next, print out the created dictionary to see the words and their numerical values! 

You should see every word in our training set and its index value. Note that the word "the" only appears once because our vocabulary only includes *unique* words.

In [20]:
# Print out the created dictionary
print(f"Word to index dictionary, must have a single occurrence for each word from the training dataset:\n{word2idx}")

Word to index dictionary, must have a single occurrence for each word from the training dataset:
{'the': 0, 'cat': 1, 'ate': 2, 'cheese': 3, 'she': 4, 'read': 5, 'that': 6, 'book': 7, 'dog': 8, 'loves': 9, 'art': 10, 'elephant': 11, 'answers': 12, 'phone': 13}


In [21]:
# A helper function for converting a sequence of words to a Tensor of numerical values
# will be used later in training
def prepare_sequence(seq: List[str], to_idx: Dict[str, int]) -> torch.Tensor:
    '''This function takes in a sequence of words and returns a 
    corresponding Tensor of numerical values (indices for each word).'''
    idxs: List[int] = [to_idx[w] for w in seq]
    idxs: np.ndarray = np.array(idxs)
    return torch.from_numpy(idxs)

In [23]:
# check out what prepare_sequence does for one of our training sentences:
example_input: torch.Tensor = prepare_sequence(seq="The dog answers the phone".lower().split(), to_idx=word2idx)
print(example_input)

tensor([ 0,  8, 12,  0, 13], dtype=torch.int32)


## Creating the Model
---

Our model will assume a few things:
1. Our input is broken down into a sequence of words, so a sentence will be [w1, w2, ...]
2. These words come from a larger list of words that we already know (a vocabulary)
3. We have a limited set of tags, `[NN, V, DET]`, which mean: a noun, a verb, and a determinant (words like "the" or "that"), respectively
4. We want to predict\* a tag for each input word

\* To do the prediction, we will pass an LSTM over a test sentence and apply a softmax function to the hidden state of the LSTM; the result is a vector of tag scores from which we can get the predicted tag for a word based on the *maximum* value in this distribution of tag scores. 

Mathematically, we can represent any tag prediction $\hat{y}_i$ as: 

$\hat{y}_i$ = $argmax_j (log Softmax(Ah_i + b))_j$

Where $A$ is a learned weight and $b$, a learned bias term, and the hidden state at timestep $i$ is $h_i$. 


### Word embeddings

We know that an LSTM takes in an expected input size and hidden size, but sentences are rarely of a consistent size, so how can we define the input of our LSTM?

Well, at the very start of this net, we'll create an `Embedding` layer that takes in the size of our vocabulary and returns a vector of a specified size, `embedding_dim`, for each word in an input sequence of words. It's important that this be the first layer in this net. You can read more about this embedding layer in [the PyTorch documentation](https://pytorch.org/tutorials/beginner/nlp/word_embeddings_tutorial.html#word-embeddings-in-pytorch).

Pictured below is the expected architecture for this tagger model.

<img src='images/speech_tagger.png' height=60% width=60% >


In [31]:
# Define LSTM nwtwork model
class LSTMTagger(nn.Module):

    # First embedding layer to ensure, that the input word 
    # tensor at each time step has the consistent size.
    word_embeddings: nn.Embedding
    # Defines the LSTM layer encapsulating the hidden state.
    lstm: nn.LSTM
    # Defines a linear layer implementing the log softamx 
    # to convert short term memory or out to class scores.
    hidden2tag: nn.Linear
    # Consistent length of word tensors at each input step.
    hidden_size: int
    # Hidden state for the LSTM layer. Keeps track of short 
    # and long term memories.
    hidden: Tuple[torch.Tensor, torch.Tensor]
    batch_size: int

    def __init__(self, embedding_dim: int, hidden_size: int, vocab_size: int, target_size: int, batch_size: int = 1) -> None:
        """Initializes model state.
        :param embedding_dim: Intended size of embedding vector. Same as the input size for 
            LSTM layer at each time step.
        :type embedding_dim: int
        :param hidden_size: Size of the hidden state for LSTM. Analogous to the number of 
            hidden neurons at each LSTM time step.
        :type hidden_size: int
        :param vocab_size: Size of the dictionary for embedding. Needed for initializing 
            the embedding layer.
        :type vocab_size: int
        :param target_size: Number of expected class scores. Same as the out features size 
            of the linear layer converting short term memory from LSTM to class scores at 
            each time step.
        :type target_size: int
        :param batch_size: Batch size for sentences as training examples.
        :type batch_size: int
        """
        super(LSTMTagger, self).__init__()
        
        self.batch_size = batch_size
        self.hidden_size = hidden_size

        # embedding layer that turns words into a vector of a specified size
        self.word_embeddings = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim)

        # the LSTM takes embedded word vectors (of a specified size) as inputs 
        # and outputs hidden states of size hidden_dim
        self.lstm = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_size)

        # the linear layer that maps the hidden state output dimension 
        # to the number of tags we want as output, tagset_size (in this case this is 3 tags)
        self.hidden2tag = nn.Linear(in_features=hidden_size, out_features=target_size)
        
        # initialize the hidden state (see code below)
        self.hidden = self.init_hidden()

        
    def init_hidden(self) -> Tuple[torch.Tensor, torch.Tensor]:
        """At the start of training, we need to initialize a hidden state;
        there will be none because the hidden state is formed based on perviously seen data.
        So, this function defines a hidden state with all zeroes and of a specified size.
        """
        # The axes dimensions are (n_layers, batch_size, hidden_dim)
        return (torch.zeros(1, 1, self.hidden_size).to(device=Device),
                torch.zeros(1, 1, self.hidden_size).to(device=Device))

    def forward(self, sentence: torch.Tensor) -> torch.Tensor:
        """Defines the feed forward behavior.
        We input the sentence with all the time steps together like 
        the second example of using the LSTM layer in the previous 
        notebook.
        """
        # Create embedded word vectors for each word in a sentence
        embedded: torch.Tensor = self.word_embeddings(sentence)
        
        # Get the output and hidden state by passing the LSTM over our word embeddings
        # the LSTM takes in our embeddings and hiddent state. The hidden state is 
        # initialized with zeros. Our batch size is 1.
        lstm_out, self.hidden = self.lstm(
            embedded.view(len(sentence), self.batch_size, -1), self.hidden)
        
        # Get the scores for the most likely tag for a word. One confusion may arise 
        # here, that we have initialized the linear layer to have the input features 
        # size same as the hidden size. When we are reshaping the lstm_out like below 
        # how it is maintained, that the shape guarantee would be maintained. Actually 
        # in this case, that happens because we have assumed the hidden size and input 
        # size to be same and in order to enforce, that we have used word embedding.
        # For instance, if embedded has the shape (5, 1, 6) including the batch_size, 
        # then lstm_out would also have the shape (5, 1, 6), on which we can then squeeze 
        # the middle dimension and use for computing log softmax.
        tag_outputs: torch.Tensor = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores: torch.Tensor = F.log_softmax(tag_outputs, dim=1)
        
        return tag_scores

## Define how the model trains

To train the model, we have to instantiate it and define the loss and optimizers that we want to use.

First, we define the size of our word embeddings. The `EMBEDDING_DIM` defines the size of our word vectors for our simple vocabulary and training set; we will keep them small so we can see how the weights change as we train.

**Note: the embedding dimension for a complex dataset will usually be much larger, around 64, 128, or 256 dimensional.**


#### Loss and Optimization

Since our LSTM outputs a series of tag scores with a softmax layer, we will use `NLLLoss`. In tandem with a softmax layer, NLL Loss creates the kind of cross entropy loss that we typically use for analyzing a distribution of class scores. We could have used `CrossEntropyLoss` as well but since we have already defined the `log_softamx` ourselves, using `NLLoss` makes sense. We'll use standard gradient descent optimization, but you are encouraged to play around with other optimizers!

In [32]:
# the embedding dimension defines the size of our word vectors
# for our simple vocabulary and training set, we will keep these small
EMBEDDING_DIM: int = 6
HIDDEN_SIZE: int = 6

# Instantiate our model and load to 
# CUDA.
model: LSTMTagger = LSTMTagger(
    embedding_dim=EMBEDDING_DIM, 
    hidden_size=HIDDEN_SIZE, 
    vocab_size=len(word2idx), 
    target_size=len(tag2idx)
).to(device=Device)

# define our loss and optimizer
loss_function: nn.NLLLoss = nn.NLLLoss()
optimizer: optim.SGD = optim.SGD(model.parameters(), lr=0.1)


Just to check that our model has learned something, let's first look at the scores for a sample test sentence *before* our model is trained. **Note that the test sentence *must* be made of words from our vocabulary otherwise its words cannot be turned into indices**.

The scores should be Tensors of length 3 (for each of our tags) and there should be scores for each word in the input sentence.

For the test sentence, "The cheese loves the elephant", we know that this has the tags (DET, NN, V, DET, NN) or `[0, 1, 2, 0, 1]`, but our network does not yet know this. In fact, in this case, our model starts out with a hidden state of all zeroes and so all the scores and the predicted tags should be low, random, and about what you'd expect for a network that is not yet trained!

In [35]:
test_sentence: List[str] = "The cheese loves the elephant".lower().split()

# see what the scores are before training
# element [i,j] of the output is the *score* for tag j for word i.
# to check the initial accuracy of our model, we don't need to train, so we use model.eval()
inputs: torch.Tensor = prepare_sequence(test_sentence, word2idx)
model.eval()
# We need to remember to load the input tensor to CUDA
tag_scores: torch.Tensor = model(inputs.to(device=Device))
print(tag_scores)

# tag_scores outputs a vector of tag scores for each word in an inpit sentence
# to get the most likely tag index, we grab the index with the maximum score!
# recall that these numbers correspond to tag2idx = {"DET": 0, "NN": 1, "V": 2}
_, predicted_tags = torch.max(tag_scores.cpu(), 1)
print('\n')
print('Predicted tags: \n',predicted_tags)

tensor([[-0.8404, -1.8030, -0.9072],
        [-0.8657, -1.7955, -0.8838],
        [-0.8825, -1.7909, -0.8688],
        [-0.8645, -1.7615, -0.8990],
        [-0.8763, -1.7772, -0.8805]], device='cuda:0',
       grad_fn=<LogSoftmaxBackward0>)


Predicted tags: 
 tensor([0, 0, 2, 0, 0])


---
## Train the Model

Loop through all our training data for multiple epochs (again we are using a small epoch value for this simple training data). This loop:

1. Prepares our model for training by zero-ing the gradients
2. Initializes the hidden state of our LSTM
3. Prepares our data for training
4. Runs a forward pass on our inputs to get tag_scores
5. Calculates the loss between tag_scores and the true tag
6. Updates the weights of our model using backpropagation

In this example, we are printing out the average epoch loss, every 20 epochs; you should see it decrease over time.

In [39]:
# normally these epochs take a lot longer 
# but with our toy data (only 3 sentences), we can do many epochs in a short time
n_epochs: int = 300

model.train()

for epoch in range(n_epochs):
    
    epoch_loss: float = 0.0
    
    # get all sentences and corresponding tags in the training data
    # Since we are processing one sentence at a time our batch size is 1.
    for sentence, tags in training_data:
        
        # zero the gradients
        model.zero_grad()

        # zero the hidden state of the LSTM, this detaches it from its history
        # IMPORTANT: This is only needed because we did our testing before and 
        # ad that point the hidden state was initialized and modified once. In 
        # order to give ourselves a clean slate we need to reinitialize the 
        # hidden state. Usually this is not required as it already happens during 
        # the initialization of the model.
        model.hidden = model.init_hidden()  # <- This is usually not required

        # prepare the inputs from the dataset for processing by out network, 
        # turn all sentences and targets into Tensors of numerical indices
        sentence_in: torch.Tensor = prepare_sequence(sentence, word2idx)
        # prepare the ground truth from te dataset in the same way
        # IMPORTANT: In order for this to work we have to convert the datatype 
        # of the ground truth to LongTensor. Because, Embedding layer works 
        # with that data type. CUDA needs all the datatypes to be consistent.
        targets: torch.Tensor = prepare_sequence(tags, tag2idx).type(torch.LongTensor)  # <- This is important for CUDA

        # forward pass to get tag scores
        tag_scores: torch.Tensor = model(sentence_in.to(device=Device))

        # compute the loss, and gradients 
        loss: torch.Tensor = loss_function(tag_scores, targets.to(device=Device))

        epoch_loss += loss.cpu().item()
        
        # Propagate backward to compute gradients
        loss.backward()
        
        # update the model parameters with optimizer.step()
        optimizer.step()
        
    # print out avg loss per 20 epochs
    if(epoch%20 == 19):
        print("Epoch: %d, loss: %1.5f" % (epoch+1, epoch_loss/len(training_data)))


Epoch: 20, loss: 0.94861
Epoch: 40, loss: 0.72423
Epoch: 60, loss: 0.51686
Epoch: 80, loss: 0.30964
Epoch: 100, loss: 0.15810
Epoch: 120, loss: 0.09109
Epoch: 140, loss: 0.05951
Epoch: 160, loss: 0.04253
Epoch: 180, loss: 0.03246
Epoch: 200, loss: 0.02598
Epoch: 220, loss: 0.02152
Epoch: 240, loss: 0.01830
Epoch: 260, loss: 0.01588
Epoch: 280, loss: 0.01399
Epoch: 300, loss: 0.01249


## Testing

See how your model performs *after* training. Compare this output with the scores from before training, above.

Again, for the test sentence, "The cheese loves the elephant", we know that this has the tags (DET, NN, V, DET, NN) or `[0, 1, 2, 0, 1]`. Let's see if our model has learned to find these tags!

In [41]:
test_sentence: List[str] = "The cheese loves the elephant".lower().split()

# see what the scores are after training
input: torch.Tensor = prepare_sequence(test_sentence, word2idx)
model.eval()
tag_scores: torch.Tensor = model(input.to(device=Device))
print(tag_scores)

# print the most likely tag index, by grabbing the index with the maximum score!
# recall that these numbers correspond to tag2idx = {"DET": 0, "NN": 1, "V": 2}
_, predicted_tags = torch.max(tag_scores, 1)
print('\n')
print('Predicted tags: \n',predicted_tags)

tensor([[-1.0442e-02, -4.6424e+00, -7.1905e+00],
        [-6.5866e+00, -4.5614e-03, -5.7533e+00],
        [-6.8559e+00, -4.0470e+00, -1.8701e-02],
        [-8.2205e-03, -4.9494e+00, -6.8129e+00],
        [-6.4957e+00, -3.7958e-03, -6.0842e+00]], device='cuda:0',
       grad_fn=<LogSoftmaxBackward0>)


Predicted tags: 
 tensor([0, 1, 2, 0, 1], device='cuda:0')


## Great job!

To improve this model, see if you can add sentences to this model and create a more robust speech tagger. Try to initialize the hidden state in a different way or play around with the optimizers and see if you can decrease model loss even faster.